In [4]:
import numpy as np

data = np.random.rand(2, 3, 4)
data

array([[[0.63932955, 0.16378762, 0.7171133 , 0.48494436],
        [0.18327753, 0.98102349, 0.75441629, 0.20908332],
        [0.36792976, 0.39509936, 0.93146659, 0.65353971]],

       [[0.57270396, 0.02729458, 0.98429982, 0.98101834],
        [0.28497553, 0.39316811, 0.91985554, 0.13473598],
        [0.84414774, 0.52212343, 0.21205006, 0.87837114]]])

In [5]:
import pyarrow as pa

tensor = pa.Tensor.from_numpy(data)
with pa.OSFile('data.arrow', 'wb') as sink:
    pa.ipc.write_tensor(tensor, sink)

In [6]:
with pa.OSFile('data.arrow', 'rb') as source:
    tensor = pa.ipc.read_tensor(source)
tensor.to_numpy()

array([[[0.63932955, 0.16378762, 0.7171133 , 0.48494436],
        [0.18327753, 0.98102349, 0.75441629, 0.20908332],
        [0.36792976, 0.39509936, 0.93146659, 0.65353971]],

       [[0.57270396, 0.02729458, 0.98429982, 0.98101834],
        [0.28497553, 0.39316811, 0.91985554, 0.13473598],
        [0.84414774, 0.52212343, 0.21205006, 0.87837114]]])

In [33]:
from operator import index
import pandas as pd
import pyarrow as pa
import numpy as np

np_array = np.array([1, 2, 3, 4], np.float32)
# np_array = np.array([[1, 2, 3, 4], [3, 4, 5, 6]], np.float32)

tensor = pa.Tensor.from_numpy(np_array)
# tensor = pa.array(np_array)
print(tensor)

data = {'col1': tensor, index: ['a', 'b']}
df = pd.DataFrame(data)
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True],
                   'four': [tensor, tensor, tensor]},
                   index=list('abc'))
df

<pyarrow.Tensor>
type: float
shape: (4,)
strides: (4,)


,one,two,three,four
a,-1.0,foo,True,"<pyarrow.Tensor>\ntype: float\nshape: (4,)\nst..."
b,NaN,bar,False,"<pyarrow.Tensor>\ntype: float\nshape: (4,)\nst..."
c,2.5,baz,True,"<pyarrow.Tensor>\ntype: float\nshape: (4,)\nst..."


In [34]:
# df.to_parquet('data.parquet') # pyarrow.lib.Tensor: did not recognize Python value type when inferring an Arrow data type

table = pa.Table.from_pandas(df)

import pyarrow.parquet as pq

pq.write_table(table, 'example.parquet')


ArrowInvalid: ('Could not convert <pyarrow.Tensor>\ntype: float\nshape: (4,)\nstrides: (4,) with type pyarrow.lib.Tensor: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column four with type object')